In [ ]:
import numpy as np, pandas as pd, datetime as dt
import matplotlib.pyplot as plt

from pathlib import Path
path = Path("/kaggle/input/h-and-m-personalized-fashion-recommendations/")

from PIL import Image
def show_images(article_ids, cols=1, texts=[], suptitle=''):
    if isinstance(article_ids, int) or isinstance(article_ids, str):
        article_ids = [article_ids]
    rows = (len(article_ids) // cols) + 1
    plt.figure(figsize=(3 + 3.5 * cols, 3 + 5 * rows))
    for i, article_id in enumerate(article_ids):
        article_id = ("0" + str(article_id))[-10:]
        text = '' if len(texts) <= i else ('\n' + texts[i])
        plt.subplot(rows, cols, i + 1)
        plt.axis('off')
        plt.title(f"{article_id}{text}", fontsize=16)
        try:
            image = Image.open(f"/kaggle/input/h-and-m-personalized-fashion-recommendations/images/{article_id[:3]}/{article_id}.jpg")
            plt.imshow(image)
        except:
            pass
    if suptitle != '': plt.suptitle(suptitle, fontsize=36, fontweight='bold')
    plt.tight_layout()

items = pd.read_csv(path / 'articles.csv')
train = pd.read_parquet('../input/hm-parquets-of-datasets/transactions_train.parquet')
adf = pd.read_parquet('../input/hm-parquets-of-datasets/articles.parquet')

def show_best_sellings(query, nitem, ncol, title, dat=train):
    usecols = ["article_id", "prod_name", "colour_group_name"]
    temp = dat.query(query).groupby('article_id').t_dat.count()\
        .reset_index().sort_values('t_dat', ascending=False).head(nitem)[['article_id']]\
        .merge(items[usecols], on='article_id', how='left').rename(columns={'t_dat':'weekly_sales_count'})
    show_images(temp.article_id.tolist(),
                ncol,
                [', '.join([p, c]) for p, c in zip(temp.prod_name.tolist(), temp.colour_group_name.tolist())],
                suptitle=title)

# **Best Selling Items : Weekly**

In [ ]:
for target_week in range(104, 92, -1):
    init_date = train[train.week == target_week].t_dat.min()
    last_date = train[train.week == target_week].t_dat.max()
    show_best_sellings(f'week == {target_week}',
                       nitem=12, ncol=(6 if 104 == target_week else 12),
                       title=f'Popular Items in Week {target_week}  :  {str(init_date)[:10]}  ---  {str(last_date)[:10]}\n')

# **Best Selling Items : Autumn 2019**

In [ ]:
for target_week in range(55, 49, -1):
    init_date = train[train.week == target_week].t_dat.min()
    last_date = train[train.week == target_week].t_dat.max()
    show_best_sellings(f'week == {target_week}',
                       nitem=12, ncol=(6 if 52 <= target_week <= 53 else 12),
                       title=f'Popular Items in Week {target_week}  :  {str(init_date)[:10]}  ---  {str(last_date)[:10]}\n')

# **Index Groups**

In [ ]:
dat = train[train.week == 104].merge(items, on='article_id', how='left')

colname = 'index_group_name'
pop_groups = dat.groupby(colname).t_dat.count().nlargest(5).index.tolist()
for group in pop_groups:
    show_best_sellings(f"{colname} == '{group}'",
                       nitem=12, ncol=(6 if (group == pop_groups[0]) else 12),
                       title=f'Index Groups : {group}\n', dat=dat)

# **Popular Sections**

In [ ]:
colname = 'section_name'
pop_groups = dat.groupby(colname).t_dat.count().nlargest(6).index.tolist()
for group in pop_groups:
    show_best_sellings(f"{colname} == '{group}'",
                       nitem=12, ncol=(6 if (group == pop_groups[0]) else 12),
                       title=f'Popular Sections : {group}\n', dat=dat)

# **Popular Colours**

In [ ]:
colname = 'perceived_colour_master_name'
pop_groups = dat.groupby(colname).t_dat.count().nlargest(5).index.tolist()
for group in pop_groups:
    show_best_sellings(f"{colname} == '{group}'",
                       nitem=12, ncol=(6 if (group == pop_groups[0]) else 12),
                       title=f'Popular Colours : {group}\n', dat=dat)

# **Popular Departments**

In [ ]:
colname = 'department_name'
pop_groups = dat.groupby(colname).t_dat.count().nlargest(10).index.tolist()
for group in pop_groups:
    show_best_sellings(f"{colname} == '{group}'",
                       nitem=12, ncol=(6 if (group == pop_groups[0]) else 12),
                       title=f'Popular Departments : {group}\n', dat=dat)